In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
np.random.seed(3)

n_classes = 3
n_examples = 15
gt = np.random.randint(n_classes, size=(3,n_examples))
pred = np.random.randint(n_classes, size=(3,n_examples))

In [3]:
print('Ground Truth: ', gt)
print('Prediction  : ', pred)

Ground Truth:  [[2 0 1 0 0 0 1 1 2 1 1 2 1 2 0]
 [0 0 1 2 0 2 1 2 2 2 2 0 1 2 1]
 [0 2 1 1 2 0 2 0 1 0 1 0 2 1 1]]
Prediction  :  [[2 0 0 2 0 0 1 2 0 1 0 0 1 1 2]
 [1 1 1 1 1 2 0 0 0 0 0 0 2 2 0]
 [1 1 2 0 1 1 0 1 0 2 2 0 0 2 0]]


## Use Sklearn
The `confusion_matrix` function from sklearn package only accepts input arrays with rank 1, so we have to flatten the inputs to get it work.

In [4]:
def iou_v1(gt, pred):
    assert gt.shape == pred.shape, "Shape not equal!"
    gt = gt.ravel() 
    pred = pred.ravel()
    cm = confusion_matrix(y_true=gt, y_pred=pred)
    # denom = (TP + FN) + (TP + FP)
    denom = np.sum(cm, axis=0) + np.sum(cm, axis=1)
    idx = range(cm.shape[0])
    # nom = TP
    nom = cm[idx, idx]
    iou = nom / (denom - nom)
    miou = np.mean(iou)
    return iou, miou

In [5]:
# cm = confusion_matrix(gt.ravel(), pred.ravel())
# print(cm)

iou1, miou1 = iou_v1(gt, pred)
print('IoU :', iou1)
print('mIoU:', miou1)

IoU : [ 0.17241379  0.15384615  0.13043478]
mIoU: 0.152231576519


## Use Tensorflow
There are two approaches to compute mIoU in tensorflow.
- Approach one: first compute the confusion matrix and then calculate the iou manually
- Approach two: use the more advanced function `mean_iou()` from `tf.metrics` module

If you're to use the first approach, you also need to flatten the input tensor before feeding to the `tf.confusion_matrix()` function. The good thing with approach one is that you can get back the IoU for each class together with the average IoU(mIoU). But if you faver the second one, life becomes much easier since the function deals with all the heavy lifting. The only thing to remember is to first run the update operation before you retrieve the value of mIoU. Functions bellow show how to use these two approaches to calculate mIoU metric and code can be easily integrated into your own project.

In [6]:
import tensorflow as tf

In [7]:
def iou_v2(gt, pred):
    with tf.Graph().as_default():
        gt = tf.convert_to_tensor(gt, dtype=tf.int32, name='gt')
        pred = tf.convert_to_tensor(pred, dtype=tf.int32, name='pred')
        flag = tf.equal(tf.shape(gt), tf.shape(pred))
        gt = tf.reshape(gt, shape=(-1,))
        pred = tf.reshape(pred, shape=(-1,))
        cm = tf.confusion_matrix(labels=gt, predictions=pred)
        denom = tf.reduce_sum(cm, axis=0) + tf.reduce_sum(cm, axis=1)
        nom = tf.diag_part(cm)
        iou = nom / (denom - nom)
        miou = tf.reduce_mean(iou)
        with tf.Session() as sess:
            assert np.all(sess.run(flag)), 'Shape not equal!'
            iou, miou = sess.run([iou, miou])
        return iou, miou
    
    
def iou_v3(gt, pred):
    with tf.Graph().as_default():
        gt = tf.convert_to_tensor(gt, dtype=tf.int32, name='gt')
        pred = tf.convert_to_tensor(pred, dtype=tf.int32, name='pred')
        flag = tf.equal(tf.shape(gt), tf.shape(pred))
        miou, update_op = tf.metrics.mean_iou(labels=gt, predictions=pred, num_classes=n_classes)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())
            assert np.all(sess.run(flag)), 'Shape not equal!'
            sess.run(update_op)
            miou = sess.run(miou)
            # miou, _ = sess.run([miou, update_op])  <<ERROR>> need to update it before retrieving the result
        return miou

In [8]:
iou2, miou2 = iou_v2(gt, pred)
assert (np.all(iou2==iou1) and miou2 == miou1), "<<ERROR>>"
print('IoU :', iou2)
print('mIoU:', miou2)

IoU : [ 0.17241379  0.15384615  0.13043478]
mIoU: 0.152231576519


In [9]:
miou3 = iou_v3(gt, pred)
#assert (miou3 == miou1), "<<ERROR>>"
print('mIoU:', miou3)

mIoU: 0.152232
